In [1]:
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

import sys

# In case you run this without installing the package, you need to add the path to the package

# This is for launching from root folder path
sys.path.append('./ex_fuzzy/')
sys.path.append('./ex_fuzzy/ex_fuzzy/')

# This is for launching from Demos folder
sys.path.append('../ex_fuzzy/')
sys.path.append('../ex_fuzzy/ex_fuzzy/')


import ex_fuzzy.fuzzy_sets as fs
import ex_fuzzy.evolutionary_fit as GA
import ex_fuzzy.utils as  utils
import ex_fuzzy.eval_tools as eval_tools
import ex_fuzzy.persistence as persistence
import ex_fuzzy.vis_rules as vis_rules

In [2]:
runner = 1 # 1: single thread, 2+: corresponding multi-thread

n_gen = 50
n_pop = 30
    
nRules = 15
nAnts = 4
vl = 3
tolerance = 0.01
fz_type_studied = fs.FUZZY_SETS.t1

In [3]:
# Import some data to play with
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [4]:
# Compute the fuzzy partitions using 3 quartiles
precomputed_partitions = utils.construct_partitions(X, fz_type_studied)

In [5]:
# We create a FRBC with the precomputed partitions and the specified fuzzy set type, 
fl_classifier = GA.BaseFuzzyRulesClassifier(nRules=nRules, linguistic_variables=precomputed_partitions, nAnts=nAnts, 
                                            n_linguistic_variables=vl, fuzzy_type=fz_type_studied, verbose=True, tolerance=tolerance, runner=runner)
fl_classifier.fit(X_train, y_train, n_gen=n_gen, pop_size=n_pop, checkpoints=0)

# Evaluate the performance of the rule base
eval_tools.eval_fuzzy_model(fl_classifier, X_train, y_train, X_test, y_test, 
                        plot_rules=False, print_rules=True, plot_partitions=False)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       30 |  0.8401278271 |  0.5259302110
     2 |       60 |  0.6527024856 |  0.5187005400
     3 |       90 |  0.5762599673 |  0.4073096971
     4 |      120 |  0.5315823895 |  0.4073096971
     5 |      150 |  0.5123477777 |  0.4073096971
     6 |      180 |  0.4937815455 |  0.4073096971
     7 |      210 |  0.4565437908 |  0.4073096971
     8 |      240 |  0.4112181013 |  0.4073096971
     9 |      270 |  0.4073096971 |  0.4073096971
    10 |      300 |  0.4073096971 |  0.4073096971
    11 |      330 |  0.4073096971 |  0.4073096971
    12 |      360 |  0.4068649003 |  0.3939657925
    13 |      390 |  0.4050857130 |  0.3939657925
    14 |      420 |  0.4007377939 |  0.3939657925
    15 |      450 |  0.3939657925 |  0.3939657925
    16 |      480 |  0.3939657925 |  0.3939657925
    17 |      510 |  0.3916942112 |  0.3258183533
    18 |      540 |  0.3848794672 |  0.3258183533
    19 |      570 |  0.3617597553 |  0.3137013865


In [6]:
# Use the rule base as a candidate to further optimize the rules
frbc = fl_classifier.rule_base

refined_classifier = GA.BaseFuzzyRulesClassifier(verbose=False, tolerance=tolerance, runner=runner, linguistic_variables=None)
refined_classifier.fit(X_train, y_train, n_gen=n_gen, pop_size=n_pop, checkpoints=0, initial_rules=frbc)

# Evaluate the performance of the rule base
eval_tools.eval_fuzzy_model(fl_classifier, X_train, y_train, X_test, y_test, 
                        plot_rules=False, print_rules=True, plot_partitions=False)

------------
ACCURACY
Train performance: 0.72
Test performance: 0.76
------------
MATTHEW CORRCOEF
Train performance: 0.6357858494852648
Test performance: 0.6643994991958752
------------
Rules for consequent: 0
----------------
IF sepal width (cm) IS High AND petal length (cm) IS Low WITH DS 0.702673796791444, ACC 1.0

Rules for consequent: 1
----------------
IF petal width (cm) IS Medium WITH DS 0.8373158104340899, ACC 0.6
IF petal length (cm) IS Medium WITH DS 0.8498017751239918, ACC 0.8484848484848485

Rules for consequent: 2
----------------
IF sepal width (cm) IS Low WITH DS 0.1417233560090703, ACC 0.8888888888888888


